In [40]:
import numpy as np
import pandas as pd
import os
import re
import requests
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as plt
import plotly_express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import os
import json

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [61]:
#Import Cleaned Data
df = pd.read_csv('data\Apps.csv')
df

,Unnamed: 0,App Name,Installs
0,0,Gakondo,10
1,1,Ampere Battery Info,5000
2,2,Vibook,50
3,3,Smart City Trichy Public Service Vehicles 17UC...,10
4,4,GROW.me,100
...,...,...,...
799890,799890,WiFi File Transfer Pro,100000
799891,799891,Disco Flashlight,1000
799892,799892,Guess the Football Player Team || Football Quiz,100
799893,799893,Deus Hand,1


In [47]:
# VARIABLE REMOVIDA 
# print(df.columns)
# df = df.drop(['App Name'], axis = 1)
# print(df.columns)

#REDUCIR EL NÚMERO DE CLASES
df.loc[df['Installs'].isin(['0 - 100', '100 - 500', '500 - 1,000']), 'Installs'] = '0 - 1,000'
df.loc[df['Installs'].isin(['1,000 - 5,000', '5,000 - 10,000']), 'Installs'] = '1,000 - 10,000'
df.loc[df['Installs'].isin(['10,000 - 50,000', '50,000 - 100,000']), 'Installs']  = '10,000 - 100,000'
df.loc[df['Installs'].isin(['100,000 - 500,000', '500,000 - 1,000,000']), 'Installs']  = '100,000 - 1,000,000'
df.loc[df['Installs'].isin(['1,000,000 - 5,000,000', '5,000,000 - 10,000,000']), 'Installs']  = '1,000,000 - 10,000,000'
df.loc[df['Installs'].isin(['10,000,000 - 50,000,000', '50,000,000 - 100,000,000']), 'Installs']  = '10,000,000 - 100,000,000'
df.loc[df['Installs'].isin(['100,000,000 - 500,000,000', '500,000,000 - 1,000,000,000']), 'Installs']  = '100,000,000 - 1,000,000,000'
df.loc[df['Installs'].isin(['1,000,000,000 - 5,000,000,000', '5,000,000,000+']), 'Installs']  = '1,000,000,000+'


df.Installs = pd.Categorical(df.Installs, ['0 - 1,000','1,000 - 10,000', '10,000 - 100,000', '100,000 - 1,000,000', '1,000,000 - 10,000,000', '10,000,000 - 100,000,000', '100,000,000 - 1,000,000,000', '1,000,000,000+'])
print(df.Installs.value_counts().sort_index())
print(df.shape)


#One hot encoding 
strat = df.PriceRange.values
df = pd.get_dummies(df, columns=['Minimum Installs', 'Maximum Installs', 'Rating Count'], drop_first=True)
print(df.columns)

#ENTRENANDO TEST SPLIT 
X = df.drop(['Installs'], axis = 1).values
y = df.Installs.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = strat, random_state = 42)
X.shape[0] == y.shape[0]


### CONSTRUYENDO LOS MODELOS ###


from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
param_grid = {"n_estimators": [50, 100, 200], "max_depth": [20, 30, 40, 50], "max_features": ['sqrt']}


grid_search = GridSearchCV(rf, param_grid=param_grid)
grid_search.fit(X, y)

print("Best score is {}".format(grid_search.best_params_))
print("Best score is {}".format(grid_search.best_score_))

#Gradient Boosting Classifier
param_grid = {
    "learning_rate": [0.01,0.025, 0.05],
    "max_depth":[5,8,10],
    "n_estimators":[25, 50, 100],
    "verbose":[1]
    }

grid_search = GridSearchCV(gbm, param_grid=param_grid)
grid_search.fit(X, y)


print("Best score is {}".format(grid_search.best_params_))
print("Best score is {}".format(grid_search.best_score_))





# MMODELO DECISION TREES
dt = DecisionTreeClassifier(max_depth = 8, max_features = 'sqrt')
dt.fit(X_train, y_train)
y_pred = dt.predict(X_train)

acc = accuracy_score(y_train, y_pred)
print("Decision Tree train data accuracy: {:.2f}".format(acc))

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Decision Tree Test data accuracy: {:.2f}".format(acc))


a = np.mean(cross_val_score(dt, X, y, scoring = 'accuracy', cv = 10))
print("Decision Tree cross validation accuracy: {:.2f}".format(a))


#RandomForest Classifier Train
rf = RandomForestClassifier(max_depth = 20, n_estimators = 100)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_train)

acc = accuracy_score(y_train, y_pred)
print("Random Forest Train data accuracy: {:.2f}".format(acc))


#RandomForest Classifier Test 
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Random Forest Test data accuracy: {:.5f}".format(acc))

#RandomForest Cross Val 
b= np.mean(cross_val_score(rf, X, y, scoring = 'accuracy', cv = 10))
print("Random Forest Cross validation accuracy : {:.2f}".format(b))



#GBM
gbm = GradientBoostingClassifier(learning_rate = 0.05, max_depth = 8, n_estimators = 400, verbose =1)
gbm.fit(X_train,y_train)

y_pred = gbm.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Gradient Boosting Classifier Test data accuracy: {:.5f}".format(acc))



#MODELO KNN
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
print('knn train data accuracy',knn.score(X_train, y_train))  
print('knn test data accuracy', knn.score(X_test,y_test))
print('knn cross validation accuracy', np.mean(cross_val_score(knn,X, y, cv = 5)))



classifiers = [('Random Forest', rf), ('Gradient Boosting Classifier', gbm)]
vc = VotingClassifier(estimators = classifiers)
vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Voting Classifier Ensemble: {:.5f}".format(acc))



0 - 1,000                      0
1,000 - 10,000                 0
10,000 - 100,000               0
100,000 - 1,000,000            0
1,000,000 - 10,000,000         0
10,000,000 - 100,000,000       0
100,000,000 - 1,000,000,000    0
1,000,000,000+                 0
Name: Installs, dtype: int64
(799895, 3)


AttributeError: 'DataFrame' object has no attribute 'PriceRange'

In [62]:
#El mejor modelo es el de DECISION TREES


# Dividir los datos en características (X) y etiquetas (y)
X = df2.drop('App Name', axis=1)
y = df2['Installs']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el clasificador de árbol de decisión
clf = DecisionTreeClassifier()

# Entrenar el modelo
clf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo: {:.2f}%".format(accuracy * 100))

Precisión del modelo: 100.00%


In [63]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

X_train.info()

Dependiente = df2['Installs'] #Selecciono la varianle destino

Independiente = df2.drop('Installs', axis=1)

X = df2["Installs"] # Variables independientes 

y = df2["App Name"]#  Variable dependiente

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

Y_pred = logreg.predict(X_test)

acc_log = round(logreg.score(X_test, y_test) * 100, 2)

acc_log

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559926 entries, 312074 to 128037
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   Unnamed: 0  559926 non-null  int64
 1   Installs    559926 non-null  int64
dtypes: int64(2)
memory usage: 12.8 MB


c:\Users\israe\OneDrive\Escritorio\Data Analist\Modulo0\BootCamp\venvdata_analytics\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


21.88

In [64]:
#Librerías
from sklearn.ensemble import RandomForestRegressor
#Inicializamos modelo y parámetros
n_estimators = 200 #Estimadores del modelo
criterion = 'squared_error' #Forma de calcular el error
max_depth = None #Límite de profundidad de los árboles
min_samples_split = 2 #Criterio de parada de profundidad
verbose = 1 #Información devuelta por el método
#Instanciamos el modelo
model = RandomForestRegressor(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split, verbose=verbose)
#Entrenamos modelo y elaboramos predicciones
model.fit(X_train, y_train)
mse_RF = np.sum((y_test-model.predict(X_test))**2)/y_test.shape[0]
mse_RF

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  2.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    1.7s finished


0.0

In [65]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
plt.figure(figsize=(100,100))
clf=DecisionTreeRegressor().fit(X_train,y_train)
plot_tree(clf,filled=True)
plt.show()

In [45]:
#MODELO KNN

# Dividir los datos en características (X) y etiquetas (y)
X = df2.drop('App Name', axis=1)
y = df2['Installs']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el clasificador KNN
knn = KNeighborsClassifier(n_neighbors=3)

# Entrenar el modelo
knn.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = knn.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo: {:.2f}%".format(accuracy * 100))

Precisión del modelo: 91.87%


In [67]:
#MODELO SVM

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
# Cargar el conjunto de datos de ejemplo
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Crear un clasificador SVM
clf = svm.SVC(kernel='linear')

# Entrenar el clasificador utilizando el conjunto de entrenamiento
clf.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Calcular la precisión del modelo
accuracy = clf.score(X_test, y_test)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.9777777777777777
